In [113]:
import json
import os
path = os.getcwd()
print(path)

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
stop_words = set(stopwords.words('english'))
nltk.download('punkt')

/Users/kalevkim/TampereUniversityLecture/BigDataEntity/Project


[nltk_data] Downloading package punkt to /Users/kalevkim/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [114]:
# Opening JSON file
f1 = open(f'{path}/data/entityCol1.json')
f2 = open(f'{path}/data/entityCol2.json')
 
# returns JSON object as
# a dictionary
ec1 = json.load(f1)
ec2 = json.load(f2)

In [115]:
def stem_clean(arr):
    joint_words = ' '.join(arr)
    val = word_tokenize(joint_words)
    valTok = list(set([w for w in val if not w.lower() in stop_words]))
    return valTok
    

In [131]:
#get all unique key and val on both EC
allAttE1 = []
allValE1 = []
dictE1 = {}
x = 1
for d in ec1:
    values_EP = []
    for key, val in (d.items()):
        allAttE1.append(key) #allKey
        treatedVal = str(val).lower()
        values_EP.append(treatedVal)
        allValE1.append(treatedVal) #allVal - treat number as string
    values_EP = stem_clean(values_EP)
    dictE1[x] = values_EP
    x = x+1

attE1Keys = list(set(allAttE1))
valE1Tokens = list(set(stem_clean(allValE1)))

461


In [128]:
valE1Tokens = stem_clean(valE1Tokens)
len(set(valE1Tokens))

461

In [132]:
allAttE2 = [] 
allValE2 = []
dictE2 = {}
x = 101
for d in ec2:
    values_EP = []
    for key, val in (d.items()):
        allAttE2.append(key) #allKey
        treatedVal = str(val).lower()
        values_EP.append(treatedVal)
        allValE2.append(treatedVal) #allVal - treat number as string
    values_EP = stem_clean(values_EP)
    dictE2[x] = values_EP
    x = x+1



attE2Keys = list(set(allAttE2))
valE2Tokens = list(set(stem_clean(allValE2)))

In [138]:
x = [w for w in valE1Tokens if w in dictE1[1]]

True

In [120]:
#stemming

#1. Lower all the letters. Done in the previous iterations

joint_words = ' '.join(valE1)
valE1 = word_tokenize(joint_words)
valE1Tok = [w for w in valE1 if not w.lower() in stop_words]

joint_words = ' '.join(valE2)
valE2 = word_tokenize(joint_words)
valE2Tok = [w for w in valE2 if not w.lower() in stop_words]


In [121]:
valE1Tokens

['lareina lang',
 'anika j. craig',
 'dolor quam pc',
 '75',
 'ut ipsum corp.',
 '14',
 'pellentesque eget corp.',
 'nibh limited',
 'iliana d. cortez',
 'chase alford',
 'louis hendrix',
 '87',
 'vance richmond',
 '62',
 '74',
 'et rutrum non corp.',
 '57',
 '60',
 'chanda moses',
 'kai w. frost',
 'a odio consulting',
 '95',
 'vestibulum ut pc',
 '88',
 'quam a felis industries',
 'gretchen e. richardson',
 'amal r. valentine',
 'orci lacus vestibulum institute',
 'non institute',
 'emerson leonard',
 'molly shields',
 '63',
 'non quam pellentesque institute',
 'risus pc',
 '72',
 'felis adipiscing fringilla corp.',
 'pellentesque a institute',
 'blossom foreman',
 'theodore henry',
 'pamela kent',
 'ac metus institute',
 'isabelle w. woodward',
 '82',
 '35',
 'nec malesuada company',
 'finland',
 'sigourney morin',
 'sodales nisi magna limited',
 'dui corporation',
 'kalia monroe',
 'ut dolor limited',
 '76',
 '16',
 'rutrum fusce associates',
 '45',
 '5',
 'bianca e. brock',
 '68',